In [27]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [28]:
import os
import pandas as pd

# LSTM_Data 폴더의 모든 CSV 파일 병합
def load_csvs_to_dataframe(folder_path):
    """
    폴더 및 하위 폴더에 있는 모든 CSV 파일을 로드하여 하나의 DataFrame으로 병합합니다.

    Args:
        folder_path (str): CSV 파일이 저장된 폴더 경로.

    Returns:
        pd.DataFrame: 병합된 DataFrame.
    """
    csv_files = []

    # 하위 폴더까지 탐색
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_path = os.path.join(root, file)

                # 클래스 번호 추출 (상위 폴더 이름에서)
                class_folder = os.path.basename(os.path.dirname(csv_path))
                try:
                    action_class = int(class_folder)
                except ValueError:
                    print(f"경고: {csv_path}의 상위 폴더 '{class_folder}'는 유효한 클래스가 아닙니다. 건너뜁니다.")
                    continue

                # CSV 불러오기 및 클래스 열 추가
                df = pd.read_csv(csv_path)
                if 'action_class' not in df.columns:
                    df['action_class'] = action_class  # 없으면 클래스 열 추가

                csv_files.append(df)

    # 모든 CSV 파일 병합
    if csv_files:
        combined_df = pd.concat(csv_files, ignore_index=True)
        print(f"총 {len(csv_files)}개의 CSV 파일을 병합했습니다. 데이터 크기: {combined_df.shape}")
        return combined_df
    else:
        print("병합할 CSV 파일이 없습니다.")
        return pd.DataFrame()


In [29]:
# 데이터 시퀀스별로 변환
def reshape_to_sequences(data, labels, seq_length):
    """
    데이터를 시퀀스 형태로 변환합니다.
    Args:
        data (np.array): 키포인트 데이터.
        labels (np.array): 레이블 데이터.
        seq_length (int): 시퀀스 길이.
    Returns:
        np.array, np.array: 시퀀스화된 입력 데이터와 레이블.
    """
    sequences = []
    sequence_labels = []
    for i in range(len(data) - seq_length + 1):
        sequences.append(data[i:i + seq_length])
        sequence_labels.append(labels[i + seq_length - 1])  # 시퀀스의 마지막 레이블 사용
    return np.array(sequences), np.array(sequence_labels)

In [30]:
# 데이터 불러오기 및 병합
folder_path = './LSTM_Data'  # 데이터 폴더 경로
df = load_csvs_to_dataframe(folder_path)

총 49개의 CSV 파일을 병합했습니다. 데이터 크기: (1470, 35)


In [31]:
# X, y 분리
X = df.iloc[:, :-1].values # 키포인트
y = df.iloc[:, -1].values # 클래스

In [32]:
# 레이블 원-핫 인코딩
y = to_categorical(y)

In [33]:
# 데이터 시퀀스 길이 지정
seq_length = 3  # 시퀀스 길이

# 데이터를 시퀀스 형태로 변환
X_seq, y_seq = reshape_to_sequences(X, y, seq_length)

In [34]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [35]:
# LSTM 모델 정의
model = Sequential([
    LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True),
    LSTM(64),
    Dense(32, activation="relu"),
    Dense(y_train.shape[1], activation="softmax")
    ])

c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
 # 모델 컴파일 및 학습
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, mode = 'auto')
# model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=300, batch_size=32, callbacks=es)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=300, batch_size=32)

Epoch 1/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6286 - loss: 0.8671 - val_accuracy: 0.6327 - val_loss: 0.8417
Epoch 2/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6552 - loss: 0.7993 - val_accuracy: 0.6599 - val_loss: 0.8128
Epoch 3/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6958 - loss: 0.7733 - val_accuracy: 0.6122 - val_loss: 0.8309
Epoch 4/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6643 - loss: 0.7449 - val_accuracy: 0.6905 - val_loss: 0.7537
Epoch 5/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6806 - loss: 0.7241 - val_accuracy: 0.6939 - val_loss: 0.7885
Epoch 6/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7426 - loss: 0.6632 - val_accuracy: 0.7075 - val_loss: 0.7766
Epoch 7/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7101 - loss: 0.7030 - val_accuracy: 0.7109 - val_loss: 0.7355
Epoch 8/300
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7379 - loss: 0.6270 - val_accuracy: 0.6395 - 

In [38]:
# 모델 저장
output_model = './Model/LSTM.h5'
model.save(output_model)

In [43]:
# 학습된 모델 평가
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"검증 데이터 정확도: {accuracy * 100:.2f}%")

검증 데이터 정확도: 76.87%
